In [15]:
import coco_text
import os
import matplotlib.pyplot as plt
import pylab
import skimage.io as io
from PIL import Image
from tqdm import tqdm

In [2]:
## coco text dataset
ct = coco_text.COCO_Text('/Data/FoodDetection/data/text_recognition/COCO/cocotext.v2.json')
data_path = '/Data/FoodDetection/data/text_recognition/COCO/train2014'
# dataType = 'train2014'

loading annotations into memory...
0:00:01.809614
creating index...
index created!


In [3]:
# get index of image including any kind of text 
img_idx = ct.getImgIds(imgIds = ct.train, catIds=[('legibility', 'legible')])

In [25]:
os.makedirs('/Data/FoodDetection/data/text_recognition/English/coco', exist_ok=True)
cant_save=[]
for idx in tqdm(img_idx):
    img = ct.loadImgs(idx)[0]
    image_path = os.path.join(data_path, img['file_name'])
    image = io.imread(image_path)  # image load
    ann_idx = ct.getAnnIds(imgIds = img['id'])
    anns = ct.loadAnns(ann_idx)  # annotation info load
    
    for idx, ann in enumerate(anns):
        if ann['utf8_string'] =='':
            continue

        top_left_x = abs(int(ann['bbox'][0] ))
        top_left_y = abs(int(ann['bbox'][1] ))
        width = abs(int(ann['bbox'][2]*1.1)) # 1.1 for residual
        height = abs(int(ann['bbox'][3]*1.1))
        cropped_image = image[top_left_y : top_left_y + height , top_left_x : top_left_x + width, :]
    
        if (cropped_image.shape[0] < 20) | (cropped_image.shape[1] < 20) : # any side under pixel 10 is hard to recognize -> drop
            continue
        img_type = Image.fromarray(cropped_image)
        try :
            img_type.save(os.path.join('/Data/FoodDetection/data/text_recognition/English/coco', f'{ann["utf8_string"]}.jpg'))
        except:
            cant_save.append([img_type, ann['utf8_string']])
            continue

  3%|▎         | 349/13880 [00:07<06:13, 36.20it/s]


KeyboardInterrupt: 

In [29]:
cant_save[1][0]

In [20]:
# plt.figure(figsize=(35, 90))
for idx, ann in enumerate(anns):
    if ann['utf8_string'] =='':
        continue
    top_left_x = abs(int(ann['bbox'][0] ))
    top_left_y = abs(int(ann['bbox'][1] ))
    width = abs(int(ann['bbox'][2]*1.1))
    height = abs(int(ann['bbox'][3]*1.1))
    cropped_image = image[top_left_y : top_left_y + height , top_left_x : top_left_x + width, :]
    
    if (cropped_image.shape[0] < 20) | (cropped_image.shape[1] < 20) :
        continue
        
    ############## visualize part #####################
    n_rows = len(anns)%4 if len(anns)//4==0 else len(anns)//4
    plt.subplot(((idx+1e-10)//n_rows)+1, 4, (idx%4)+1 )
    plt.imshow(cropped_image)
    plt.xlabel(f"{ann['utf8_string']}\nsize : {cropped_image.shape}")
plt.show()